In [262]:
import pandas as pd
import numpy as np
import calendar

# чтобы видеть все столбцы
pd.set_option('display.max_columns', 100)

# Чтение файла

In [263]:
date='2022.09.09'
folder=date+'/'

# Названия столбцов для ЖЗУ
col_names=['№ дела/акта','Страхователь','№ полиса','первичный/возобновление',
           'Застрахованный объект','Собственник пострадавшего имущества',
           'Пострадавший объект','ПВУ','пву - тип', 'Дата начала полиса',
           'Дата окончания полиса','Дата страхового случая',
           'Дата поступления заявления о СС','Страховой случай',
           'Дата закрытия','Дата отказа','Страховая сумма по риску','Вал-юта',
           'Сумма заявленного убытка','Вал-юта', 'Поле1', 'Поле2',
           'Тип обращения', 'Заявитель','Выгодоприобретатель']

xl = pd.ExcelFile(folder+'жзу.xls')
zhzu = pd.DataFrame()
for sheet in xl.sheet_names:
    sheet=pd.read_excel(xl,sheet,header=None,names=col_names,
                        dtype={'Премия':float},parse_dates=date_columns)
    zhzu = pd.concat([zhzu,sheet])
    
zhzu=zhzu[5:-2].drop_duplicates(subset=['№ дела/акта','№ полиса'])

# Даты для ЖЗУ
date_columns = ['Дата начала полиса', 'Дата окончания полиса',
                'Дата страхового случая', 'Дата поступления заявления о СС', 
                'Дата закрытия', 'Дата отказа']

for col in date_columns:
#     zhzu[col]=zhzu[col].astype(object).where(zhzu[col].notnull(), None)
    zhzu[col]=pd.to_datetime(zhzu[col])

# PreProcessing

## Обрезка по условиям + контроль кол-ва пустот

In [264]:
nans=zhzu[zhzu['Дата поступления заявления о СС'].isna()].shape[0]
if nans<=10:
    zhzu=zhzu[zhzu['Дата поступления заявления о СС'].notna()]
    print('ВСЕ ОК!')
else: print('Проблема! Много пустых:',nans)

shifr=pd.read_excel('классификаторы шифров.xlsx',dtype={'шифр': float})
zhzu['шифр']=zhzu['№ полиса'].astype(str).str[:3].astype(float)
zhzu=zhzu.merge(shifr,how='left',on='шифр')

zhzu=zhzu[(zhzu['Линия бизнеса']!='02 - ОСАГО')|
          ((zhzu['Линия бизнеса']=='02 - ОСАГО')&(zhzu['пву - тип']!='Прям'))]

ВСЕ ОК!


## Проставление недели месяца и года

In [265]:
#Месяц и год для сводной аут данных на текущий момент
normizer=lambda dt: '0'+dt if len(dt)==1 else dt
zhzu.loc[:,'год заявления']=zhzu['Дата поступления заявления о СС'].dt.year
zhzu.loc[:,'год заявления']=zhzu['год заявления'].astype(str)
zhzu.loc[:,'месяц заявления']=zhzu['Дата поступления заявления о СС'].dt.month
zhzu.loc[:,'месяц заявления']=zhzu['месяц заявления'].astype(str).apply(normizer)

In [266]:
#Данная функция ставит номер недели в месяце
#с учётом, что неделя считается с сб по пт
#и неделя принадлежит тому месяцу, которому принадлежит ее пт
#Если данный день в переменной принадлежит первой недели в этом месяце,
#то будет 1, к i добавляется 1, так как i - это индекс, то есть N-1
def order_in_time_maker(dt):
    mth = calendar.monthcalendar(dt.year, dt.month)
    #Распознаем номер недели
    #Если день находится в неделе, где нет в конце пустых дней,
    #то ставим номер недели
    #иначе ставим 1, то есть начало в следующем месяце
    for iwk, wk in enumerate(mth):
        if dt.day in wk:
            if wk[-1]!=0: iwk=iwk+1; mth=dt.month; year=dt.year; break
            elif dt.month==12: iwk=1; mth=1; year=dt.year+1; break
            else: iwk=1; mth=dt.month+1; year=dt.year; break
        else: pass
    return [str(year),str(mth),str(iwk)]

#Обозначаем, что неделя начинается с сб (по логике алгоритма предсказания)
calendar.setfirstweekday(calendar.SATURDAY)

In [267]:
#Ставим номер недели
zhzu.loc[:,'субботы']=zhzu['Дата поступления заявления о СС']
zhzu.loc[:,'субботы']=zhzu['субботы'].apply(order_in_time_maker)
#Ставим месяц и год
zhzu.loc[:,'месяц по субботам']=zhzu['субботы'].str[1].apply(normizer)
zhzu.loc[:,'год по субботам']=zhzu['субботы'].str[0]
#Ставим номер недели+месяц+год
zhzu.loc[:,'субботы']=zhzu['год по субботам']+zhzu['месяц по субботам']+\
                      zhzu['субботы'].str[2]

zhzu.loc[:,'субботы']=zhzu.loc[:,'субботы'].astype(int)
#Создадим фрейм для нумерации недель подряд а не блоками в месяц
#Отчет ведем с 04.01.2020 (суббота неделя вторая в месяце!)
week_number_df=zhzu.loc[zhzu['субботы']>=2020012,'субботы'].unique()
week_number_df.sort()
week_number_df=pd.DataFrame(week_number_df,columns=['субботы'])
week_number_df.loc[:,'неделя по субботам']=[i for i in range(1,len(week_number_df)+1)]
#Проставим новый номер недели в zhzu
zhzu=zhzu.merge(week_number_df,how='left',on='субботы')
zhzu=zhzu[zhzu['неделя по субботам'].notna()]
zhzu.loc[:,'неделя по субботам']=zhzu['неделя по субботам'].astype(int)

## Формирование сводных

In [268]:
index=['год по субботам','месяц по субботам','неделя по субботам']

pivot_sum_sub=zhzu.pivot_table('Сумма заявленного убытка',columns='Линия бизнеса',
                               index=index, aggfunc='sum')

pivot_count_sub=zhzu.pivot_table('№ дела/акта', columns='Линия бизнеса',
                                 index=index, aggfunc='count')

pivot_sum_sub=pivot_sum_sub.reset_index().sort_values(index)
pivot_count_sub=pivot_count_sub.reset_index().sort_values(index)

index=['год заявления','месяц заявления']

pivot_sum_real=zhzu.pivot_table('Сумма заявленного убытка', columns='Линия бизнеса',
                           index=index, aggfunc='sum')

pivot_count_real=zhzu.pivot_table('№ дела/акта', columns='Линия бизнеса',
                             index=index, aggfunc='count')

pivot_sum_real=pivot_sum_real.reset_index().sort_values(index)
pivot_count_real=pivot_count_real.reset_index().sort_values(index)

# Прогнозирование

## Формирование данных о суммах по ЛБ

In [269]:
# index=['неделя по субботам','месяц по субботам','год по субботам']
# index=index+pivot_sum_sub.columns[3:].values.tolist()
# pivot_sum_sub=pivot_sum_sub[index]
pivot_sum_sub.loc[:,'месяц по субботам']=pivot_sum_sub['месяц по субботам'].astype(int)
pivot_sum_sub.loc[:,'год по субботам']=pivot_sum_sub['год по субботам'].astype(int)
dataset_sum=pivot_sum_sub.replace(np.nan,0)

In [270]:
dataset_mod_sum=dataset_sum.iloc[:,[0,1,2,3]]
dataset_cmtpl_sum=dataset_sum.iloc[:,[0,1,2,4]]
dataset_imushestvo_sum=dataset_sum.iloc[:,[0,1,2,6]]
dataset_otv_sum=dataset_sum.iloc[:,[0,1,2,7]]
dataset_gruz_sum=dataset_sum.iloc[:,[0,1,2,8]]
dataset_vzr_sum=dataset_sum.iloc[:,[0,1,2,9]]
dataset_ns_sum=dataset_sum.iloc[:,[0,1,2,10]]
dataset_ipoteka_sum=dataset_sum.iloc[:,[0,1,2,11]]
dataset_other_sum=dataset_sum.iloc[:,[0,1,2,12]]

#КАСКО
x_mod_sum=dataset_mod_sum.iloc[:,[0,1,2]]
y_mod_sum=dataset_mod_sum.iloc[:, 3]

#ОСАГО
x_cmtpl_sum=dataset_cmtpl_sum.iloc[:,[0,1,2]]
y_cmtpl_sum=dataset_cmtpl_sum.iloc[:, 3]

#имущество
x_imushestvo_sum=dataset_imushestvo_sum.iloc[:,[0,1,2]]
y_imushestvo_sum=dataset_imushestvo_sum.iloc[:, 3]

#ответственность
x_otv_sum=dataset_otv_sum.iloc[:,[0,1,2]]
y_otv_sum=dataset_otv_sum.iloc[:, 3]

#грузы
x_gruz_sum=dataset_gruz_sum.iloc[:,[0,1,2]]
y_gruz_sum=dataset_gruz_sum.iloc[:, 3]

#взр
x_vzr_sum=dataset_vzr_sum.iloc[:,[0,1,2]]
y_vzr_sum=dataset_vzr_sum.iloc[:, 3]

#НС
x_ns_sum=dataset_ns_sum.iloc[:,[0,1,2]]
y_ns_sum=dataset_ns_sum.iloc[:, 3]

#ипотека
x_ipoteka_sum=dataset_ipoteka_sum.iloc[:,[0,1,2]]
y_ipoteka_sum=dataset_ipoteka_sum.iloc[:, 3]

#прочее
x_other_sum=dataset_other_sum.iloc[:,[0,1,2]]
y_other_sum=dataset_other_sum.iloc[:, 3]

## Формирование данных о количествах по ЛБ

In [271]:
# index=['неделя по субботам','месяц по субботам','год по субботам']
# index=index+pivot_count_sub.columns[3:].values.tolist()
# pivot_count_sub=pivot_count_sub[index]
pivot_count_sub.loc[:,'месяц по субботам']=pivot_count_sub['месяц по субботам'].astype(int)
pivot_count_sub.loc[:,'год по субботам']=pivot_count_sub['год по субботам'].astype(int)
dataset_count=pivot_count_sub.replace(np.nan,0)

In [272]:
dataset_mod_count=dataset_count.iloc[:,[0,1,2,3]]
dataset_cmtpl_count=dataset_count.iloc[:,[0,1,2,4]]
dataset_imushestvo_count=dataset_count.iloc[:,[0,1,2,6]]
dataset_otv_count=dataset_count.iloc[:,[0,1,2,7]]
dataset_gruz_count=dataset_count.iloc[:,[0,1,2,8]]
dataset_vzr_count=dataset_count.iloc[:,[0,1,2,9]]
dataset_ns_count=dataset_count.iloc[:,[0,1,2,10]]
dataset_ipoteka_count=dataset_count.iloc[:,[0,1,2,11]]
dataset_other_count=dataset_count.iloc[:,[0,1,2,12]]

#КАСКО
x_mod_count=dataset_mod_count.iloc[:,[0,1,2]]
y_mod_count=dataset_mod_count.iloc[:, 3]

#ОСАГО
x_cmtpl_count=dataset_cmtpl_count.iloc[:,[0,1,2]]
y_cmtpl_count=dataset_cmtpl_count.iloc[:, 3]

#имущество
x_imushestvo_count=dataset_imushestvo_count.iloc[:,[0,1,2]]
y_imushestvo_count=dataset_imushestvo_count.iloc[:, 3]

#ответственность
x_otv_count=dataset_otv_count.iloc[:,[0,1,2]]
y_otv_count=dataset_otv_count.iloc[:, 3]

#грузы
x_gruz_count=dataset_gruz_count.iloc[:,[0,1,2]]
y_gruz_count=dataset_gruz_count.iloc[:, 3]

#взр
x_vzr_count=dataset_vzr_count.iloc[:,[0,1,2]]
y_vzr_count=dataset_vzr_count.iloc[:, 3]

#НС
x_ns_count=dataset_ns_count.iloc[:,[0,1,2]]
y_ns_count=dataset_ns_count.iloc[:, 3]

#ипотека
x_ipoteka_count=dataset_ipoteka_count.iloc[:,[0,1,2]]
y_ipoteka_count=dataset_ipoteka_count.iloc[:, 3]

#прочее
x_other_count=dataset_other_count.iloc[:,[0,1,2]]
y_other_count=dataset_other_count.iloc[:, 3]

In [ ]:
Делаю массив недель для прогнозного МЕСЯЦА

In [291]:
#Дата сегодня
# today=pd.Timestamp('today')
#Для проверок функции
today=pd.to_datetime('30-09-2022',format='%d-%m-%Y')
#Массив недель этого месяца
mth=calendar.monthcalendar(today.year, today.month)
if today.day==mth[-1][-1]:
    if today.month==12: mth=calendar.monthcalendar(today.year+1, 1)
    else: mth=calendar.monthcalendar(today.year, today.month+1)
else:
    for iwk, wk in enumerate(mth):
        
        if today.day in wk:
            if wk[-1]!=0: iwk=iwk+1; month=today.month; year=today.year; break
            elif today.month==12: iwk=iwk+1; month=1; year=today.year+1; break
            else: iwk=iwk+1; month=today.month+1; year=today.year; break
        else: pass

In [292]:
mth

[[1, 2, 3, 4, 5, 6, 7],
 [8, 9, 10, 11, 12, 13, 14],
 [15, 16, 17, 18, 19, 20, 21],
 [22, 23, 24, 25, 26, 27, 28],
 [29, 30, 31, 0, 0, 0, 0]]

In [282]:
#Дата сегодня
# today=pd.Timestamp('today')
#Для проверок функции
today=pd.to_datetime('01-10-2022',format='%d-%m-%Y')
#Массив недель этого месяца
mth=calendar.monthcalendar(today.year, today.month)
# if today.day==mth[-1][-1]:
#     if today.month==12: mth=calendar.monthcalendar(today.year+1, 1)
#     else: mth=calendar.monthcalendar(today.year, today.month+1)
    
# else: pass
#Определим номер текущей недели
for iwk, wk in enumerate(mth):
    if today.day in wk:
        [0,0,0,0,0,0,0]
        
        if wk[-1]!=0: iwk=iwk+1; month=today.month; year=today.year; break
        elif today.month==12: iwk=iwk+1; month=1; year=today.year+1; break
        else: iwk=iwk+1; month=today.month+1; year=today.year; break
    else: pass
#Определим номер следующей недели в прогнозе
next_wk=zhzu.loc[zhzu['неделя по субботам'].notna(),
                 'неделя по субботам'].astype('int').max()+1

x_test=[]
#Каждую оставшуюся неделю в месяце добавим для прогноза
for wk in range(next_wk,next_wk+len(mth)-iwk):
    x_test.append([year,month,wk])
x_test=pd.DataFrame(x_test,columns=['год по субботам','месяц по субботам',
                                    'неделя по субботам'])

In [285]:
#Дата сегодня
# today=pd.Timestamp('today')
#Для проверок функции
today=pd.to_datetime('01-10-2022',format='%d-%m-%Y')
#Массив недель этого месяца
mth=calendar.monthcalendar(today.year, today.month)
if today.day==mth[-1][-1]:
    if today.month==12:
        today=pd.to_datetime('01-'+'01-'+str(today.year+1),format='%d-%m-%Y')
        mth=calendar.monthcalendar(today.year, today.month)
    else:
        today=pd.to_datetime('01-'+str(today.month+1)+'-'+str(today.year),format='%d-%m-%Y')
        mth=calendar.monthcalendar(today.year, today.month)
else: pass


In [ ]:
order_in_time_maker(dt)

In [286]:
mth

[[1, 2, 3, 4, 5, 6, 7],
 [8, 9, 10, 11, 12, 13, 14],
 [15, 16, 17, 18, 19, 20, 21],
 [22, 23, 24, 25, 26, 27, 28],
 [29, 30, 31, 0, 0, 0, 0]]

In [283]:
x_test

,год по субботам,месяц по субботам,неделя по субботам
0,2022,10,141
1,2022,10,142
2,2022,10,143
3,2022,10,144


## Прогнозирование с помощью ML

In [274]:
from lightgbm import LGBMRegressor
model_mod_sum = LGBMRegressor()
model_mod_sum.fit(x_mod_sum,y_mod_sum)
y_pred_mod_sum=model_mod_sum.predict(x_test)

model_mod_count = LGBMRegressor()
model_mod_count.fit(x_mod_count,y_mod_count)
y_pred_mod_count=model_mod_count.predict(x_test)


model_cmtpl_sum = LGBMRegressor()
model_cmtpl_sum.fit(x_cmtpl_sum,y_cmtpl_sum)
y_pred_cmtpl_sum=model_cmtpl_sum.predict(x_test)

model_cmtpl_count = LGBMRegressor()
model_cmtpl_count.fit(x_cmtpl_count,y_cmtpl_count)
y_pred_cmtpl_count=model_cmtpl_count.predict(x_test)


model_imushestvo_sum = LGBMRegressor()
model_imushestvo_sum.fit(x_imushestvo_sum,y_imushestvo_sum)
y_pred_imushestvo_sum=model_imushestvo_sum.predict(x_test)

model_imushestvo_count = LGBMRegressor()
model_imushestvo_count.fit(x_imushestvo_count,y_imushestvo_count)
y_pred_imushestvo_count=model_imushestvo_count.predict(x_test)


model_otv_sum = LGBMRegressor()
model_otv_sum.fit(x_otv_sum,y_otv_sum)
y_pred_otv_sum=model_otv_sum.predict(x_test)

model_otv_count = LGBMRegressor()
model_otv_count.fit(x_otv_count,y_otv_count)
y_pred_otv_count=model_otv_count.predict(x_test)


model_gruz_sum = LGBMRegressor()
model_gruz_sum.fit(x_gruz_sum,y_gruz_sum)
y_pred_gruz_sum=model_gruz_sum.predict(x_test)

model_gruz_count = LGBMRegressor()
model_gruz_count.fit(x_gruz_count,y_gruz_count)
y_pred_gruz_count=model_gruz_count.predict(x_test)


model_vzr_sum = LGBMRegressor()
model_vzr_sum.fit(x_vzr_sum,y_vzr_sum)
y_pred_vzr_sum=model_vzr_sum.predict(x_test)

model_vzr_count = LGBMRegressor()
model_vzr_count.fit(x_vzr_count,y_vzr_count)
y_pred_vzr_count=model_vzr_count.predict(x_test)


model_ipoteka_sum = LGBMRegressor()
model_ipoteka_sum.fit(x_ipoteka_sum,y_ipoteka_sum)
y_pred_ipoteka_sum=model_ipoteka_sum.predict(x_test)

model_ipoteka_count = LGBMRegressor()
model_ipoteka_count.fit(x_ipoteka_count,y_ipoteka_count)
y_pred_ipoteka_count=model_ipoteka_count.predict(x_test)


from xgboost import XGBRegressor
model_ns_sum = XGBRegressor()
model_ns_sum.fit(x_ns_sum,y_ns_sum)
y_pred_ns_sum=model_ns_sum.predict(x_test)

model_ns_count = XGBRegressor()
model_ns_count.fit(x_ns_count,y_ns_count)
y_pred_ns_count=model_ns_count.predict(x_test)


model_other_sum = XGBRegressor()
model_other_sum.fit(x_other_sum,y_other_sum)
y_pred_other_sum=model_other_sum.predict(x_test)

model_other_count = XGBRegressor()
model_other_count.fit(x_other_count,y_other_count)
y_pred_other_count=model_other_count.predict(x_test)

In [275]:
col_names=['01 - КАСКО','02 - ОСАГО','04 - Имущество','05 - Ответственность',
           '06 - Грузы','07 - ВЗР','08 - НС','09 - Ипотека','10 - Прочее']

sum_data=np.array([y_pred_mod_sum,y_pred_cmtpl_sum,
                   y_pred_imushestvo_sum,y_pred_otv_sum,
                   y_pred_gruz_sum,y_pred_vzr_sum,y_pred_ns_sum,
                   y_pred_ipoteka_sum,y_pred_other_sum]).T

y_pred_sum=pd.DataFrame(sum_data, columns=col_names)
y_pred_sum=pd.concat([x_test,y_pred_sum],axis=1)
pivot_sum_sub=pd.concat([pivot_sum_sub,y_pred_sum])

count_data=np.array([y_pred_mod_count,y_pred_cmtpl_count,
                   y_pred_imushestvo_count,y_pred_otv_count,
                   y_pred_gruz_count,y_pred_vzr_count,y_pred_ns_count,
                   y_pred_ipoteka_count,y_pred_other_count]).T

y_pred_count=pd.DataFrame(count_data, columns=col_names)
y_pred_count=pd.concat([x_test,y_pred_count],axis=1)
pivot_count_sub=pd.concat([pivot_count_sub,y_pred_count])

# Поменять местами!!!

In [278]:
with pd.ExcelWriter(folder+date+'_out_данные.xlsx') as writer:  
    pivot_sum_sub.to_excel(writer,sheet_name='суммы',index=False)
    pivot_count_sub.to_excel(writer,sheet_name='кол-во',index=False)
    pivot_sum_real.to_excel(writer,sheet_name='суммы по месяцам',index=False)
    pivot_count_real.to_excel(writer,sheet_name='кол-во по месяцам',index=False)

zhzu.to_excel(folder+date+'_zhzu_out.xlsx')